<h1> Week 3 Assignment (Wikipedia Dataframe) <h1>


<h2>Task 1 <h2>

In [1]:
import pandas as pd
import numpy as np
import requests 
from bs4 import BeautifulSoup

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_html = requests.get(url).text
soup = BeautifulSoup(wiki_html, 'html.parser')

**Accessing and retrieving the postal code html info**


In [3]:
data = []
for tr in soup.tbody.find_all('tr'):
    data.append([ td.get_text().strip() for td in tr.find_all('td')])

**Creating Data-Frame**

In [4]:
df = pd.DataFrame(data, columns=['PostalCode','Borough','Neighborhood'])
df.head()

,PostalCode,Borough,Neighborhood
0,None,None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


**Dropping rows with not assigned values**

In [5]:
df=df.dropna()
df = df[(df.PostalCode != 'Not assigned' ) & (df.Borough != 'Not assigned') & (df.Neighborhood != 'Not assigned')]


In [6]:
df.head()

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


**Grouping my df by PostalCode and Borough then joining similar neighbourhoods**

In [7]:
df = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.columns = ['PostalCode', 'Borough', 'Neighborhood']
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


**Save to CSV**

In [8]:
df.to_csv('task1.csv', index=False)

**Printing Shape**

In [9]:
df.shape

(102, 3)

<h2>Task 2<h2>

**Creating a New Data Frame for Task 2**

In [10]:
df2=pd.read_csv('task1.csv')

In [11]:
df2.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


**Installing GeoCoder**

In [12]:
!pip install geocoder
import geocoder

**Retrieving Longitude and Langitude Data**

In [13]:
def get_latlng(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords

**Appending to original dataframe**

In [14]:
postal_codes = df2['PostalCode']    
coordinates = [ get_latlng(postal_code) for postal_code in postal_codes.tolist() ]
df_coordinates = pd.DataFrame(coordinates, columns=['Latitude', 'Longitude'])
df2['Latitude'] = df_coordinates['Latitude']
df2['Longitude'] = df_coordinates['Longitude']

In [15]:
df2.head(12)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440
5,M1J,Scarborough,Scarborough Village,43.743125,-79.231750
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.726245,-79.263670
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.713133,-79.285055
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.723575,-79.234976
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.696665,-79.260163


**Saving the new dataframe into another csv file**

In [16]:
df2.to_csv('task2.csv', index=False)

<h2>Task 3, Exploring and Clustering Toronto <h2>

**Importing required libraries**

In [17]:
import requests 
from pandas.io.json import json_normalize # tranform JSON file into a pd df
import matplotlib.cm as cm
!conda install -c conda-forge folium=0.5.0 --yes
import folium #for mapping
import matplotlib.colors as colors #for plotting
from sklearn.cluster import KMeans #for clustering




Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be 

In [18]:
toronto_coords = get_latlng('')
t_neighborhoods = pd.read_csv('task2.csv')

**Creating a seperate df for Toronto as specified in the task**


In [19]:
toronto_data = t_neighborhoods[t_neighborhoods['Borough'].str.contains("Toronto")].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676845,-79.295225
1,M4K,East Toronto,"The Danforth West, Riverdale",43.683262,-79.355120
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.667965,-79.314673
3,M4M,East Toronto,Studio District,43.662766,-79.334830
4,M4N,Central Toronto,Lawrence Park,43.728160,-79.387085


**Mapping all the data points**

In [21]:
map_toronto = folium.Map(location=[toronto_coords[0], toronto_coords[1]], zoom_start=12)

# adding markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{} - {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

**Now I will use Foursquare to explore the neighbourhoods**

In [22]:
CLIENT_ID = 'LR34M3R5AXRGN0HHIZ4HNSO31Y11GQ3TYN1DROI4MVCGUB4I' # your Foursquare ID
CLIENT_SECRET = 'Z42DRMZKFEWODB3FOSU2CATXOJRINONTPU5VSMW44UPD0K4P' # your Foursquare Secret
VERSION = '20180605'
LIMIT = 30

The next function will get venues with names and coordinates into a df

In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

**Getting all venues and checking df**

In [24]:
t_neighborhoods = toronto_data
t_venues = getNearbyVenues(names=t_neighborhoods['Neighborhood'],
                                   latitudes=t_neighborhoods['Latitude'],
                                   longitudes=t_neighborhoods['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [25]:
print(t_venues.shape)
t_venues.head(2)

(762, 7)


,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676845,-79.295225,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676845,-79.295225,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store


**Looking at unique categories**

In [26]:
print(len(t_venues['Venue Category'].unique()))

167


**Evaluating count of venues**

In [44]:
t_venues.groupby('Neighbourhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",30,30,30,30,30,30
Berczy Park,30,30,30,30,30,30
"Brockton, Exhibition Place, Parkdale Village",30,30,30,30,30,30
Business Reply Mail Processing Centre 969 Eastern,30,30,30,30,30,30
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",30,30,30,30,30,30
"Cabbagetown, St. James Town",30,30,30,30,30,30
Central Bay Street,30,30,30,30,30,30
"Chinatown, Grange Park, Kensington Market",30,30,30,30,30,30
Christie,10,10,10,10,10,10


**Analyzing each neighbourhood for all venues**

In [27]:
# one hot encoding
t_onehot = pd.get_dummies(t_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
t_onehot['Neighbourhood'] = t_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [t_onehot.columns[-1]] + list(t_onehot.columns[:-1])
t_onehot = t_onehot[fixed_columns]

t_onehot.head()

,Neighbourhood,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**Examining df size**

In [28]:
t_onehot.shape


(762, 168)

**Now lets evaluate mean of frequencies and reorganize the dataframe by the neighbourhoods**

In [29]:
t_grouped = t_onehot.groupby('Neighbourhood').mean().reset_index()
t_grouped

,Neighbourhood,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Adelaide, King, Richmond",0.066667,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.0,...,0.000000,0.000000,0.00,0.00,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000
1,Berczy Park,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.00,0.00,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.0,...,0.000000,0.000000,0.00,0.00,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.0,...,0.000000,0.000000,0.00,0.00,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.0,...,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333
5,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Central Bay Street,0.033333,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.000000,0.0,...,0.000000,0.033333,0.00,0.00,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.1,...,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.033333,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000


**Checking shape one more time**

In [30]:
t_grouped.shape

(37, 168)

**Printing each neighborhood with the top 5 venues**

In [31]:
num_top_venues = 5
for neigh in t_grouped['Neighbourhood']:
    print("----"+neigh+"----")
    temp = t_grouped[t_grouped['Neighbourhood'] == neigh].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.13
1                 Café  0.13
2           Steakhouse  0.07
3  American Restaurant  0.07
4            Speakeasy  0.03


----Berczy Park----
                venue  freq
0            Beer Bar  0.07
1  Seafood Restaurant  0.07
2                Café  0.07
3      Farmers Market  0.07
4        Cocktail Bar  0.07


----Brockton, Exhibition Place, Parkdale Village----
                    venue  freq
0             Coffee Shop  0.10
1                   Hotel  0.07
2      Italian Restaurant  0.07
3                    Café  0.07
4  Furniture / Home Store  0.07


----Business Reply Mail Processing Centre 969 Eastern----
          venue  freq
0    Steakhouse  0.10
1          Café  0.10
2   Pizza Place  0.07
3         Hotel  0.07
4  Concert Hall  0.07


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
                  venue  freq
0    Italian Rest

**Sorting dataframe in an descending manner to get the top ones**

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# creating columns according to quantity of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = t_grouped['Neighbourhood']

for ind in np.arange(t_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(t_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.shape

(37, 11)

In [33]:
t_grouped.shape

(37, 168)

**Clustering neighborhoods with kmeans**

In [34]:
# set number of clusters
kclusters = 10

toronto_grouped_clustering = t_grouped.drop('Neighbourhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_[0:10])
print(len(kmeans.labels_))

[0 0 0 0 0 0 0 0 0 0]
37


**Dropping last row of in t_neighbourhoods to match dimensions since neighbourhoods_venues_sorted only has 36 rows since the business reply centre has no venues**

In [35]:
t_neighborhoods.drop(t_neighborhoods.index[37],inplace=True)

In [36]:
toronto_merged = t_neighborhoods

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighborhood')

toronto_merged # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676845,-79.295225,0,Health Food Store,Trail,Neighborhood,Pub,Yoga Studio,Discount Store,Farm,Falafel Restaurant,Event Space,Electronics Store
1,M4K,East Toronto,"The Danforth West, Riverdale",43.683262,-79.355120,0,Grocery Store,Fast Food Restaurant,Discount Store,Bus Line,Park,Gift Shop,Farmers Market,Gourmet Shop,Falafel Restaurant,Event Space
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.667965,-79.314673,0,Park,Sandwich Place,Food & Drink Shop,Pub,Burrito Place,Burger Joint,Liquor Store,Fast Food Restaurant,Fish & Chips Shop,Steakhouse
3,M4M,East Toronto,Studio District,43.662766,-79.334830,0,Italian Restaurant,Brewery,Diner,Arts & Crafts Store,Coffee Shop,American Restaurant,Pizza Place,Snack Place,Café,Chinese Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728160,-79.387085,0,Swim School,Bus Line,Yoga Studio,Fish Market,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Electronics Store
5,M4P,Central Toronto,Davisville North,43.712815,-79.388526,0,Gym,Hotel,Park,Breakfast Spot,Food & Drink Shop,Clothing Store,Dessert Shop,Event Space,Cupcake Shop,Dance Studio
6,M4R,Central Toronto,North Toronto West,43.714523,-79.406960,0,Park,Gym Pool,Playground,Garden,Discount Store,Farm,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant
7,M4S,Central Toronto,Davisville,43.703395,-79.385964,0,Dessert Shop,Coffee Shop,Pizza Place,Café,Sandwich Place,Italian Restaurant,Park,Indian Restaurant,Diner,Restaurant
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.690655,-79.383561,0,Gym,Playground,Tennis Court,Restaurant,Cupcake Shop,Cuban Restaurant,Farm,Falafel Restaurant,Event Space,Electronics Store
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686083,-79.402335,0,Light Rail Station,Coffee Shop,Liquor Store,Supermarket,Eastern European Restaurant,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space


**Creating map to visualize clusters**

In [41]:
# create map
map_clusters = folium.Map(location=[toronto_coords[0], toronto_coords[1]], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**Evaluating Cluster 1**

In [45]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Health Food Store,Trail,Neighborhood,Pub,Yoga Studio,Discount Store,Farm,Falafel Restaurant,Event Space,Electronics Store
1,East Toronto,0,Grocery Store,Fast Food Restaurant,Discount Store,Bus Line,Park,Gift Shop,Farmers Market,Gourmet Shop,Falafel Restaurant,Event Space
2,East Toronto,0,Park,Sandwich Place,Food & Drink Shop,Pub,Burrito Place,Burger Joint,Liquor Store,Fast Food Restaurant,Fish & Chips Shop,Steakhouse
3,East Toronto,0,Italian Restaurant,Brewery,Diner,Arts & Crafts Store,Coffee Shop,American Restaurant,Pizza Place,Snack Place,Café,Chinese Restaurant
4,Central Toronto,0,Swim School,Bus Line,Yoga Studio,Fish Market,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Electronics Store
5,Central Toronto,0,Gym,Hotel,Park,Breakfast Spot,Food & Drink Shop,Clothing Store,Dessert Shop,Event Space,Cupcake Shop,Dance Studio
6,Central Toronto,0,Park,Gym Pool,Playground,Garden,Discount Store,Farm,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant
7,Central Toronto,0,Dessert Shop,Coffee Shop,Pizza Place,Café,Sandwich Place,Italian Restaurant,Park,Indian Restaurant,Diner,Restaurant
8,Central Toronto,0,Gym,Playground,Tennis Court,Restaurant,Cupcake Shop,Cuban Restaurant,Farm,Falafel Restaurant,Event Space,Electronics Store
9,Central Toronto,0,Light Rail Station,Coffee Shop,Liquor Store,Supermarket,Eastern European Restaurant,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space


**Evaluating cluster 5**

In [46]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Downtown Toronto,4,Coffee Shop,Breakfast Spot,Gym / Fitness Center,Bakery,Mexican Restaurant,Pub,Restaurant,Electronics Store,Event Space,Italian Restaurant
